# Similarity Network Fusion  + downstream analysis

Authors: Casper de Visser (casper.devisser@radboudumc.nl)

## Introduction

This notebook contains the downstream analysis of the fused sample similarity matrix that was constructed with SNF  (Wang 2014). Spectral clustering is performed on these sample similarities and these clusters are compared with the behavioral data and phenotypic covariates.

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from sklearn.cluster import spectral_clustering
from sklearn.metrics import v_measure_score
import snf
from snf import metrics
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels
import muon
import itertools

ModuleNotFoundError: No module named 'snf'

## Read in MuData object

In [ ]:
mdata = muon.read(mudata_object)

# Load fused network

In [2]:
# Load Pandas DataFrame of fused network
fused_df = pd.read_csv(snf_df_path,  header=None)

# Convert to numpy array
fused_network = fused_df.to_numpy()

# Retrieve data type names from input file name
data_type_names = str(snf_df_path)[2:-20]
data_type_names_list = list(data_type_names.split("', '"))

## Rewrite the lists of omics names into abbreviations, to make DF values more readable

temp_list = []

for name in data_type_names_list:
    if name == 'acylcarnitines':
        temp_list.append('AC')
    if name == 'aminoacids':
        temp_list.append('AA')
    if name == 'fattyacids':
        temp_list.append('FA')
    if name == 'proteins':
        temp_list.append('PR')
    if name == 'lipidomics-neg':
        temp_list.append('LN')
    if name == 'lipidomics-pos':
        temp_list.append('LP')
    if name == 'mRNA':
        temp_list.append('MR')
    if name == 'miRNA-mature':
        temp_list.append('MI')
    if name == 'EM-seq':
        temp_list.append('EM')
    if name == 'miRNA-qRT-PCR':
        temp_list.append('RT')

NameError: name 'snf_df_path' is not defined

# Get correct samples subset 

In [ ]:
# Find common samples in omics combination
obs_list = []
for i in data_type_names_list:
    obs_list.append(list(mdata[i].obs.index))

common_obs = list(set.intersection(*map(set, obs_list)))

# Subset samples present in omics combination
muon.pp.filter_obs(mdata, common_obs)

# Heatmap showing fused network

In [ ]:
# Functions


def sort_fused_network(fused_network, labels_array):
    # Make Pandas Dataframes
    df = pd.DataFrame(fused_network)
    df_labels = pd.DataFrame(labels_array)
    df_labels.columns = ["Label"]
    # sort label df
    df_labels = df_labels.sort_values(by=['Label'])
    # sort fused network df with sorted labels
    df = df.reindex(df_labels.index)
    df = df[df_labels.index]
    array = df.to_numpy()
    np.fill_diagonal(array, 0)
    return(array)

def make_heatmap(array, n_clusters, data_types):
    # Create heatmap
    heatmap = plt.imshow(array, cmap='hot', interpolation='nearest')

    # Set axis names, title etc.
    plt.xlabel('samples')
    plt.ylabel('samples') 
    cbar = plt.colorbar(heatmap)
    cbar.ax.set_ylabel('sample correlations', loc="top")
    plt.suptitle(str('Fused network: ' +  str(data_types) + '\nNumber of clusters: {:.2f}'.format(round(float(n_clusters))))) 
    plt.show()

    return(plt)  

In [ ]:
# determine optimal number of clusters (estimated via an eigengap approach)
best, second = snf.get_n_clusters(fused_network)

# Perform spectral clustering on the fused network
labels = spectral_clustering(fused_network, n_clusters=best)
labels_second = spectral_clustering(fused_network, n_clusters=second)
    
# Sort Fused networks according to labels found by spectral clustering
sorted_fused_network_best = sort_fused_network(fused_network, labels)
sorted_fused_network_second = sort_fused_network(fused_network, labels_second)
    
from IPython.display import display, Markdown
display(Markdown("## Fused network" ))

make_heatmap(sorted_fused_network_best,  best, data_type_names)
make_heatmap(sorted_fused_network_second,  second, data_type_names)

## Add SNF labels to phenotype data (.obs)

In [ ]:
# Spectral clustering

# determine optimal number of clusters (estimated via an eigengap approach)
best, second = snf.get_n_clusters(fused_network)

# Perform spectral clustering on the fused network
labels = spectral_clustering(fused_network, n_clusters=best)
labels_second = spectral_clustering(fused_network, n_clusters=second)

# Add cluster labels from SNF

SNF_label_best_colname =  "SNF_label_" + str(best)
SNF_label_second_colname =  "SNF_label_" + str(second)

mdata.obs[SNF_label_best_colname] = labels
mdata.obs[SNF_label_second_colname] = labels_second

for i in mdata.obs.index:
    mdata.obs.at[i, SNF_label_best_colname] =  "SNF_"+ str(mdata.obs.at[i, SNF_label_best_colname])
    mdata.obs.at[i, SNF_label_second_colname] =  "SNF_"+ str(mdata.obs.at[i, SNF_label_second_colname])

### Export SNF labels to csv

In [ ]:
os.mkdir('labels')

new = mdata.obs.loc[:,[SNF_label_best_colname, SNF_label_second_colname]]



new.to_csv(str('labels/SNF_labels_' + str(temp_list) + '.csv'))

print(temp_list)

## Compare SNF clusters to categorical phenotype data (Mosaic plots)

In [ ]:
# Function to generate the mosaic plot

from statsmodels.graphics.mosaicplot import mosaic

colors = ['#e69F00', '#56b4e9', '#009e73', '#f0e442', '#0072b2', '#d55e00', '#cc79a7', '#000000']

def make_mosaic(df, col1, col2, mosaic_title='Pheno variable'):
    
    #Sort df on pheno value that is plotted against the clusters
    df = df.sort_values(by=[col1])
    
    
    #Adjust plot size to value counts
    number_of_pheno_values = len(df[col1].value_counts())
    number_of_clusters = len(df[col2].value_counts())
    
    
    if number_of_pheno_values < 2:
        print('No differences are observed in this column among the subjects')
    
    else:
    
        if number_of_pheno_values < 4:
            number_of_pheno_values = 4
    
        # Figure size
        fig, ax = plt.subplots(figsize=(number_of_pheno_values*2,number_of_clusters*1.5))      
        
        
        # Figure color palette
        props= {}
        e = 0
        a =  0.6 - number_of_clusters/10 
        for i in df[col1].unique():
            for j in df[col2].unique():
                props[(str(i), str(j))] = {'color': colors[e], 'alpha' : a}
                a += (0.6 - number_of_clusters/10)
            e += 1
            a = (0.6 - number_of_clusters/10)
        
        # Figure lables (percentages)
        labels_dict={}
        for i in df[col1].unique():
            for j in df[col2].unique():
                samples = len(df[(df[col1] == i) & (df[col2] == j)])
                percentage = round(samples/len(df.index) * 100, 2)
                labels_dict[(str(i), str(j))] = str(i) + '\n' + 'SNF '+ str(j)[-1] + '\n' + str(percentage) + '%'

        # Generate plot
        mosaic(df, 
               [col1, col2], 
               ax=ax, 
               axes_label=False,
               gap=0.06,
               properties = props,
               labelizer = lambda k: labels_dict[k])
        plt.xlabel(col1, fontsize=20)
        plt.ylabel('SNF cluster', fontsize=15) # This is now hardcoded
        plt.title(mosaic_title, fontsize=15)
        plt.show()
        plt.close()

        return(plt)

In [ ]:
make_mosaic(mdata.obs, 'BMI.group', SNF_label_best_colname)

In [ ]:
make_mosaic(mdata.obs, 'BMI.group', SNF_label_second_colname)

In [ ]:
make_mosaic(mdata.obs, 'Age.group', SNF_label_best_colname)

In [ ]:
make_mosaic(mdata.obs, 'Age.group', SNF_label_second_colname)

In [ ]:
make_mosaic(mdata.obs, 'Sex', SNF_label_best_colname)

In [ ]:
make_mosaic(mdata.obs, 'Sex', SNF_label_second_colname)

In [ ]:
# Remove categorical with 0 values  TODO: do for all phenotype columns

mdata.obs['Smoking.Status'] = mdata.obs['Smoking.Status'] .cat.remove_unused_categories()

make_mosaic(mdata.obs, 'Smoking.Status', SNF_label_best_colname)

In [ ]:
make_mosaic(mdata.obs, 'Smoking.Status', SNF_label_second_colname)

In [ ]:
# Chi square testing on contingency tables

In [ ]:
import scipy.stats
from scipy.stats import chi2

categorical_vars = ['Age.group', 'Sex', 'BMI.group', 'Smoking.Status']

chisquare_best_list = []

for i in categorical_vars:
    row = []
    ct_table = pd.crosstab(mdata.obs[i], mdata.obs[SNF_label_best_colname])
    chi2_stat, p, dof, expected = scipy.stats.chi2_contingency(ct_table)
    row.append(i)
    row.append(chi2_stat)
    row.append(p)
    row.append(dof)
    chisquare_best_list.append(row)
    
a = np.array(chisquare_best_list)
df = pd.DataFrame(a[:,1:], index = a[:,0], columns = ['chi2 test', 'p_value', 'degrees of freedom'])
df

In [ ]:
chisquare_second_list = []

for i in categorical_vars:
    row = []
    ct_table = pd.crosstab(mdata.obs[i], mdata.obs[SNF_label_second_colname])
    chi2_stat, p, dof, expected = scipy.stats.chi2_contingency(ct_table)
    row.append(i)
    row.append(chi2_stat)
    row.append(p)
    row.append(dof)
    chisquare_second_list.append(row)
    
a = np.array(chisquare_second_list)
df = pd.DataFrame(a[:,1:], index = a[:,0], columns = ['chi2 test', 'p_value', 'degrees of freedom'])
df

# Compare SNF clusters with batches

In [ ]:
# Add cluster labels from SNF
for omics in data_type_names_list:
    mdata[omics].obs[SNF_label_best_colname] = labels
    mdata[omics].obs[SNF_label_second_colname] = labels_second

    for i in mdata[omics].obs.index:
        mdata[omics].obs.at[i, SNF_label_best_colname] =  "SNF_"+ str(mdata[omics].obs.at[i, SNF_label_best_colname])
        mdata[omics].obs.at[i, SNF_label_second_colname] =  "SNF_"+ str(mdata[omics].obs.at[i, SNF_label_second_colname])

In [ ]:
# Best clustering

batch_synonyms = ['Batch', 'Group', 'Parameter Value[date]', 'RT']

for omics in data_type_names_list:
    for syn in batch_synonyms:
        if syn in mdata[omics].obs.columns: 
            make_mosaic(mdata[omics].obs, syn, SNF_label_best_colname, omics)    

In [ ]:
# Second best clustering

for omics in data_type_names_list:
    for syn in batch_synonyms:
        if syn in mdata[omics].obs.columns: 
            make_mosaic(mdata[omics].obs, syn, SNF_label_second_colname, omics)   

## Chisquare tests

In [ ]:
chisquare_best_list = []

for omics in data_type_names_list:
    row = []
    for syn in batch_synonyms:
        if syn in mdata[omics].obs.columns: 
            ct_table = pd.crosstab(mdata[omics].obs[syn], mdata[omics].obs[SNF_label_best_colname])
            chi2_stat, p, dof, expected = scipy.stats.chi2_contingency(ct_table)
            row.append(omics)
            row.append(chi2_stat)
            row.append(p)
            row.append(dof)
            chisquare_best_list.append(row)
    
a = np.array(chisquare_best_list)
df = pd.DataFrame(a[:,1:], index = a[:,0], columns = ['chi2 test', 'p_value', 'degrees of freedom'])
df

In [ ]:
chisquare_second_list = []

for omics in data_type_names_list:
    row = []
    for syn in batch_synonyms:
        if syn in mdata[omics].obs.columns: 
            ct_table = pd.crosstab(mdata[omics].obs[syn], mdata[omics].obs[SNF_label_second_colname])
            chi2_stat, p, dof, expected = scipy.stats.chi2_contingency(ct_table)
            row.append(omics)
            row.append(chi2_stat)
            row.append(p)
            row.append(dof)
            chisquare_second_list.append(row)
    
a = np.array(chisquare_second_list)
df = pd.DataFrame(a[:,1:], index = a[:,0], columns = ['chi2 test', 'p_value', 'degrees of freedom'])
df

## Functions used for statistics 

In [ ]:
def make_significant_bold(x):
    bold = 'bold' if x < 0.05 else ''
    return 'font-weight: %s' % bold


def make_pvalue_table(p_value_list):
    a = np.array(p_value_list)
    df = pd.DataFrame(a[:,1:], index = a[:,0], columns = ['test statistic', 'p-value'])
    df = df.dropna()
    df = df.loc[df["p-value"] != "nan"]
    df['test statistic'] = pd.to_numeric(df['test statistic'])
    df['p-value'] = pd.to_numeric(df['p-value'])
    p_values = np.asarray(df['p-value'].values.tolist())
    corrected_p_values = statsmodels.stats.multitest.fdrcorrection(p_values)
    df['FDR corrected p-value'] = corrected_p_values[1].tolist()
    df.style.applymap(make_significant_bold)
    return(df)


def perform_statistical_testing(n_clusters, df, colname):

    snf_groups = []

    for i in range(n_clusters):
        snf_label = 'SNF_' + str(i)
        group = df[df["SNF_label_" + str(n_clusters)] == snf_label]
        snf_groups.append(group[colname])


    if len(snf_groups) > 2:
        for i in snf_groups:
            p_values = stats.kruskal(*i)
    
    else:
        p_values = stats.mannwhitneyu(snf_groups[0], snf_groups[1])
        
    return(p_values)

## Compare clusters on numerical pheno variables

In [ ]:
# Perform shapiro test for normality
shapiro = stats.shapiro(mdata.obs["BMI"])

if float(list(shapiro)[1]) < 0.05:
    print('Shapiro test shows that BMI values are not normally distributed')
    print('p-value', list(shapiro)[1])
    
else:
    print('Shapiro test shows that BMI values are normally distributed')
    print('p-value', list(shapiro)[1])

In [ ]:
# Drop sample for statiscial testing
# TODO: this sample has Nan values for blood cell counts

mdata_without_NA =  mdata.obs

if 'CZC3066' in mdata.obs.index:
    mdata_without_NA = mdata_without_NA.drop('CZC3066')

In [ ]:
phenotype_columns = ["Age", "BMI", "Leukocytes", "Erythrocytes", "Hemoglobin", "Hematocrit", "Platelets", "MCV"]
p_value_list = []

for i in phenotype_columns: #TODO: exclude Nan values
    row = []
    statistics = stats.shapiro(mdata_without_NA[i])
    row.append(i)
    row.append(list(statistics)[0])
    row.append(list(statistics)[1])
    p_value_list.append(row)
    
df = make_pvalue_table(p_value_list)
df.style.applymap(make_significant_bold)
df

In [ ]:
p_value_list = []

for i in phenotype_columns: #TODO: exclude Nan values
    row = []
    best_stat = perform_statistical_testing(best, mdata_without_NA, i)
    row.append(i + "_vs_best_clustering")
    row.append(list(best_stat)[0])
    row.append(list(best_stat)[1])
    p_value_list.append(row)
    
    row2 = []
    row2.append(i + "_vs_second_clustering")
    second_stat = perform_statistical_testing(second, mdata_without_NA, i)
    row2.append(list(second_stat)[0])
    row2.append(list(second_stat)[1])
    p_value_list.append(row2)
 

df = make_pvalue_table(p_value_list)
df.style.applymap(make_significant_bold)
df

## Test sample order with ranked Spearman test

### Batches vs SNF label

In [ ]:
p_value_list = []


batch_synonyms = ['Batch', 'Group', 'Parameter Value[date]', 'RT']

for omics in data_type_names_list:
    for syn in batch_synonyms:
        if syn in mdata[omics].obs.columns:
            mdata_input = mdata[omics].obs
            if 'CZC3066' in mdata_input.index:
                mdata_input = mdata_input.drop('CZC3066')
            row = []
            best_stat = scipy.stats.spearmanr(mdata_input[syn], mdata_input[SNF_label_best_colname])
            row.append(omics + "_vs_best_clustering")
            row.append(list(best_stat)[0])
            row.append(list(best_stat)[1])
            p_value_list.append(row)

            row2 = []
            row2.append(omics + "_vs_second_clustering")
            second_stat = scipy.stats.spearmanr(mdata_input[syn], mdata_input[SNF_label_second_colname])
            row2.append(list(second_stat)[0])
            row2.append(list(second_stat)[1])
            p_value_list.append(row2)
            
df = make_pvalue_table(p_value_list)
df.style.applymap(make_significant_bold)
df

### Plot how samples are distributed over the differerent batches in Targeted Metabolomics 

In [ ]:
column1 = mdata['acylcarnitines'].obs['Parameter Value[date]']
column2 = mdata['aminoacids'].obs['Parameter Value[date]']
column3 = mdata['fattyacids'].obs['Parameter Value[date]']
new_df = pd.concat([column1, column2, column3], axis=1, keys=['acyl', 'amino', 'fattyacids'])

make_mosaic(new_df, 'amino', 'acyl', mosaic_title='Batches')
make_mosaic(new_df, 'fattyacids', 'amino', mosaic_title='Batches')
make_mosaic(new_df, 'acyl', 'fattyacids', mosaic_title='Batches')

### Pheno covariates vs SNF label

In [ ]:
p_value_list = []

for i in phenotype_columns: #TODO: exclude Nan values
    row = []
    best_stat = scipy.stats.spearmanr(mdata_without_NA[i], mdata_without_NA[SNF_label_best_colname])
    row.append(i + "_vs_best_clustering")
    row.append(list(best_stat)[0])
    row.append(list(best_stat)[1])
    p_value_list.append(row)
    
    row2 = []
    row2.append(i + "_vs_second_clustering")
    second_stat = scipy.stats.spearmanr(mdata_without_NA[i], mdata_without_NA[SNF_label_second_colname])
    row2.append(list(second_stat)[0])
    row2.append(list(second_stat)[1])
    p_value_list.append(row2)
    

df = make_pvalue_table(p_value_list)
df.style.applymap(make_significant_bold)
df